In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px

In [4]:
df = pd.read_csv('/content/Preprocessed data.csv', encoding= 'latin1')
vader = pd.read_csv('/content/VADER.csv', encoding= 'latin1')


In [5]:
df.head()

,tweet_id,Full_text,tweet_timestamp,user_location,date,time,text_length,tokens,No_stopwords,lemmatized,joined_string,text,text_modified
0,1.348558e+18,One similarity between Paracetemol Ashwin Both...,2021-01-11 09:10:57,NaN,2021-01-11,09:10:57,9,"['One', 'similarity', 'between', 'Paracetemol'...","['One', 'similarity', 'Paracetemol', 'Ashwin',...","['One', 'similarity', 'Paracetemol', 'Ashwin',...",['One similarity Paracetemol Ashwin Both PAINE...,['One similarity Paracetemol Ashwin Both PAINE...,One similarity Paracetemol Ashwin Both PAINE k...
1,1.348558e+18,seriously ????? Is removing the batsmens guar...,2021-01-11 09:12:13,"Bengaluru South, Karnataka",2021-01-11,09:12:13,12,"['seriously', '?', '?', '?', 'Is', 'removing',...","['seriously', '?', '?', '?', 'Is', 'removing',...","['seriously', '?', '?', '?', 'Is', 'removing',...",['seriously ? ? ? Is removing batsman guard op...,['seriously ? ? ? Is removing batsman guard op...,seriously ? ? ? Is removing batsman guard oppo...
2,1.348558e+18,If Rahul Dravid is The wall then Ravichandran ...,2021-01-11 09:12:19,Halifax/Huddersfield,2021-01-11,09:12:19,21,"['If', 'Rahul', 'Dravid', 'is', 'The', 'wall',...","['If', 'Rahul', 'Dravid', 'wall', 'Ravichandra...","['If', 'Rahul', 'Dravid', 'wall', 'Ravichandra...",['If Rahul Dravid wall Ravichandran Ashwin Han...,['If Rahul Dravid The wall Ravichandran Ashwin...,If Rahul Dravid wall Ravichandran Ashwin Hanum...
3,1.348558e+18,give it back its an Ash Tag not hashtag,2021-01-11 09:12:29,Singapore,2021-01-11,09:12:29,9,"['give', 'it', 'back', 'its', 'an', 'Ash', 'Ta...","['give', 'back', 'Ash', 'Tag', 'hashtag']","['give', 'back', 'Ash', 'Tag', 'hashtag']",['give back Ash Tag hashtag'],['give back Ã¢ÂÂ Ash Tag hashtag'],give back Ash Tag hashtag
4,1.348559e+18,Let me tell U a story Once upon a time Kangar...,2021-01-11 09:12:56,Sri Lanka,2021-01-11,09:12:56,45,"['Let', 'me', 'tell', 'U', 'a', 'story', 'Once...","['Let', 'tell', 'U', 'story', 'Once', 'upon', ...","['Let', 'tell', 'U', 'story', 'Once', 'upon', ...",['Let tell U story Once upon time Kangaroos Ti...,['Let tell U story Once upon time Kangaroos Ti...,Let tell U story Once upon time Kangaroos Tige...


In [6]:
missed = pd.DataFrame()
missed['column'] = df.columns

missed['percent'] = [round(100* df[col].isnull().sum() / len(df), 2) for col in df.columns]
missed = missed.sort_values('percent')
missed = missed[missed['percent']>0]

fig = px.bar(
    missed, 
    x='percent', 
    y="column", 
    orientation='h', 
    title='Missed values percent for every column (percent > 0)', 
    height=400, 
    width=600
)

fig.show()

In [8]:
ds = df['user_location'].value_counts().reset_index()
ds.columns = ['user_location', 'count']
ds = ds[ds['user_location']!='NA']
ds = ds.sort_values(['count'])

fig = px.bar(
    ds.tail(40), 
    x="count", 
    y="user_location", 
    orientation='h', title='Top 40 user locations by number of tweets', 
    width=800, 
    height=800
)

fig.show()

In [9]:
def pie_count(data, field, percent_limit, title):
    
    data[field] = data[field].fillna('NA')
    data = data[field].value_counts().to_frame()

    total = data[field].sum()
    data['percentage'] = 100 * data[field]/total    

    percent_limit = percent_limit
    otherdata = data[data['percentage'] < percent_limit] 
    others = otherdata['percentage'].sum()  
    maindata = data[data['percentage'] >= percent_limit]

    data = maindata
    other_label = "Others(<" + str(percent_limit) + "% each)"
    data.loc[other_label] = pd.Series({field:otherdata[field].sum()}) 
    
    labels = data.index.tolist()   
    datavals = data[field].tolist()
    
    trace=go.Pie(labels=labels,values=datavals)

    layout = go.Layout(
        title = title,
        height=600,
        width=600
        )
    
    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)
    
pie_count(df, 'user_location', 0.5, 'Number of tweets per location')

In [16]:
# Tweet distribution over time in a day
df['tweet_timestamp'] = pd.to_datetime(df['tweet_timestamp'], errors='coerce')

df['hour'] = df['tweet_timestamp'].dt.hour
ds = df['hour'].value_counts().reset_index()
ds.columns = ['hour', 'count']
ds['hour'] = 'Hour ' + ds['hour'].astype(str)
fig = px.bar(
    ds, 
    x="hour", 
    y="count", 
    orientation='v', 
    title='Tweets distribution over hours', 
    width=800
)
fig.show()

In [15]:
#Now we are going to calculate the length for every tweet in dataset
df['tweet_length'] = df['Full_text'].str.len()
fig = px.histogram(
    df, 
    x="tweet_length", 
    nbins=80, 
    title='Tweet length distribution', 
    width=800,
    height=700
)
fig.show()